# Solving Transition States for Heavy Metals and Monomers Database with pyscf (REWRITE IN PYSCF)

Requires:
 - pyscf
 - numpy
 - geometric
 - https://github.com/pyscf/qsdopt


TODO:
 - look at the [solvation methods documentation](https://pyscf.org/user/solvent.html#introduction) to add consideration for the water that this system would normally be solvated in

 # check for unit consistency!

In [1]:
import os
import json
from datetime import datetime

os.environ["OMP_NUM_THREADS"] = "8"  # 8 threads
os.environ["PYSCF_MAX_MEMORY"] = "30000"  # 30 GB memory

FUNCTIONAL = "HF"  #  shortcut, eventually use 'B3LYP'
BASIS_SET = "LANL2DZ"  # shortcut, eventually use 'def2-QZVP'
LEVEL_OF_THEORY = FUNCTIONAL + "/" + BASIS_SET

Function to load the molecules as a PySCF molecule from an xyz file

In [2]:
from pyscf import gto

def get_molecule(fname):
    return gto.M(atom=fname, basis=BASIS_SET)

In [3]:
chitosan_monomer = get_molecule("structures/chitosan_monomer.xyz")
chitosan_monomer.charge, chitosan_monomer.spin = 0, 0
cadmium = get_molecule("structures/cadmium.xyz")
cadmium.charge, cadmium.spin = 2, 0
combined_starting_materials = get_molecule("structures/combined_starting_materials.xyz")
combined_starting_materials.charge, combined_starting_materials.spin = 2, 0
product = get_molecule("structures/chitosan_monomer_with_cadmium.xyz")
product.charge, product.spin = 2, 0
ts_guess = get_molecule("structures/ts_guess.xyz")
ts_guess.charge, ts_guess.spin = 2, 0

data_dict = {
    # "chitosan_monomer": {"pyscf_molecule": chitosan_monomer, "is_ts": False},
    # "cadmium": {"pyscf_molecule": cadmium, "is_ts": False},
    # "starting_materials": {"pyscf_molecule": combined_starting_materials, "is_ts": False},
    # "product": {"pyscf_molecule": product, "is_ts": False},
    "ts_guess": {"pyscf_molecule": ts_guess, "is_ts": True},
}

Optimization and Solving of Wavefunction

__later upgrade this to a DFT method__

Uses a different method for transition states as suggested in the [pyscf documentation](https://pyscf.org/user/geomopt.html#transition-state-optimization)

In [4]:
from pyscf import solvent
from pyscf.scf import HF
from pyscf.geomopt.geometric_solver import optimize
from pyscf.qsdopt.qsd_optimizer import QSD

def optfreq(mol, is_ts=False):
    mol_eq = None
    if mol.natm > 1:
        mean_field = mol.HF(  # hartree-fock
            mol,
            # xc=FUNCTIONAL, <-- add back for DFT upgrade
        ).ddCOSMO()
        if is_ts:
            optimizer = QSD(mean_field, stationary_point="TS")
            optimizer.kernel()
            mol_eq = mol
        else:
            mol_eq = optimize(mean_field, maxsteps=100, prefix=mol.output)
    else:
        # skip optimization for monatomics
        mol_eq = mol
    # solve wavefunction - re-initalize with new geometry
    mean_field_eq = mol_eq.HF(
        mol_eq,
        # xc=FUNCTIONAL, <-- add back for DFT upgrade
    ).ddCOSMO()
    mean_field_eq.kernel(dm0=None)  # No initial guess for density matrix
    return mean_field_eq

/home/jackson/mambaforge/envs/pyscf/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


Solve for the thermochemistry, see [this tutorial from pyscf's GitHub](https://github.com/pyscf/pyscf/blob/master/examples/hessian/10-thermochemistry.py)

In [5]:
from pyscf.hessian import thermo

def therm(mean_field_eq, temperature_K=298.15, pressure_Pa=101325):
    hessian = mean_field_eq.Hessian().kernel()
    freq_info = thermo.harmonic_analysis(mean_field_eq.mol, hessian)
    thermo_info = thermo.thermo(
        mean_field_eq,
        freq_info["freq_au"],
        temperature_K,
        pressure_Pa,
    )
    return thermo_info

Iterate through all the species and call the appropriate functions, saving the outputs as we go

In [6]:
for name, data in data_dict.items():
    print("~" * 50, "\n", " " * 15 + "Simulating:", name, "\n", "~" * 50, flush=True)
    data["pyscf_molecule"].output = "logfiles/" + name + "-" + datetime.now().strftime("%Y%m%d-%H%M%S") + ".log"
    data["pyscf_molecule"].verbose = 4
    data["pyscf_molecule"].build()  # make the above logging changes take affect
    mean_field_eq = optfreq(data["pyscf_molecule"], data["is_ts"])
    data["mean_field_eq"] = mean_field_eq
    thermoinfo = therm(mean_field_eq)
    data["therminfo"] = thermoinfo

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
                Simulating: ts_guess 
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
output file: logfiles/ts_guess-20231106-171127.log


/home/jackson/mambaforge/envs/pyscf/lib/python3.11/site-packages/pyscf/lib/misc.py:609: UserWarning: method set() only supports keyword arguments.
Arguments <pyscf.gto.mole.Mole object at 0x7f343ba9a810> are ignored.
  warnings.warn('method set() only supports keyword arguments.\n'


Later add IRC for the Transition States (?)

In [ ]:
print(json.dumps(data, indent=4, default=lambda _: "<not serializble>"))